In [12]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import PromptTemplate
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import DocArrayInMemorySearch
from operator import itemgetter

In [2]:
model = ChatOpenAI(model="gpt-4o-mini")
embeddings = OpenAIEmbeddings()
parser = StrOutputParser()

In [3]:
template = """
Answer the question based on the context below. If you can't 
answer the question, reply "I don't know".

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)

In [17]:
loader = PyMuPDFLoader("ref_data/TWINGO/MR413X4416A000.pdf", extract_images=True)
pages = loader.load()

In [20]:
pages[1].page_content

'16A-2\nMR-413-X44-16A000$125.mif\nV2\n16A\nЗАПУСК ДВИГАТЕЛЯ И ЗАРЯДКА \nАККУМУЛЯТОРНОЙ БАТАРЕИ\nДиагностика – Перечень и расположение элементов\n1\nСтартер \n2\nГенератор \n3\nЦЭКБС\n1\n2\nTwingo II\n3\nMR-413-X44-16A000$125.mif\n120411120518安\n121207'

In [15]:
for page_index in range(len(pages)): # iterate over pdf pages
    page = pages[page_index] # get the page
    image_list = page.extract_images()

    # print the number of images found on the page
    if image_list:
        print(f"Found {len(image_list)} images on page {page_index}")
    else:
        print("No images found on page", page_index)

AttributeError: 'Document' object has no attribute 'extract_images'

In [5]:
vectorstore = DocArrayInMemorySearch.from_documents(pages, embedding=embeddings)

c:\Users\andre\AppData\Local\Programs\Python\Python312\Lib\site-packages\pydantic\_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [6]:
retriever = vectorstore.as_retriever()

In [7]:
chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
    }
    | prompt
    | model
    | parser
)

In [15]:
question = """
Что делать если  Стартер не функционирует
"""

In [16]:
for s in chain.stream({"question": question}):
    print(s, end="", flush=True)

Если стартер не функционирует, выполните следующие шаги:

1. Проверьте состояние аккумуляторной батареи.
2. Проверьте состояние силового предохранителя.
3. Убедитесь в исправности предохранителя F9 компонента 1016.
4. Проверьте, не заклинило ли двигатель.
5. Проверьте надежность затяжки следующих наконечников:
   - аккумуляторную батарею,
   - стартера,
   - "массы" двигателя,
   - "массы" кузова.
6. Убедитесь, что клеммы надежно затянуты.
7. Проверьте исправность следующих проводов:
   - провод, соединяющий вывод "+" аккумуляторной батареи с контактом B "+" стартера,
   - провод, соединяющий вывод "-" аккумуляторной батареи с "массой" кузова автомобиля,
   - провод, соединяющий двигатель с "массой" кузова автомобиля.
8. Устраните неисправности поврежденных проводов и их наконечников. Если проводка неисправна и есть способ ремонта, устраните неисправность; если нет, замените проводку.
9. Попробуйте запустить двигатель. Если якорь стартера не вращается, проверьте, слышен ли щелчок при е